In [53]:
import pandas as pd
import numpy as np
from slugify import slugify
pd.set_option('display.max_columns', None)
pd. set_option("display.max_rows", None)
from Levenshtein import distance

## I. Selecionar os dados: determinar quais conjuntos de dados serão utilizados e documentar os motivos de inclusão/exclusão.

In [54]:
#df = pd.read_csv("../data/raw/iowa_liquor_train_test_split_sample.csv")

In [55]:
#df.head()

In [56]:
df = pd.read_csv("../data/raw/iowa_liquor_train_test_split_sample.csv", usecols=[2,10,22,23],encoding='utf-8')

In [57]:
df.columns = [slugify(col, lowercase=True, separator='_') for col in df.columns]
df.columns

Index(['date', 'county', 'sale_dollars', 'volume_sold_liters'], dtype='object')

df.head(1)

### Remoção das colunas as quais não iremos utilizar
#### Vamos deixar somente as colunas data, salle_dollars, volume_sold_liters, pois vamos realizar regressão linear em dados numéricos
#### Vamos deixar a coluna county para poder utilizar regressão linear e prever o volume em litros e o valor em dolares

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518216 entries, 0 to 2518215
Data columns (total 4 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   county              object 
 2   sale_dollars        object 
 3   volume_sold_liters  float64
dtypes: float64(1), object(3)
memory usage: 76.8+ MB


In [59]:
df.head()

,date,county,sale_dollars,volume_sold_liters
0,02/26/2013,Scott,$8.99,0.38
1,08/22/2017,WARREN,$287.94,1.75
2,08/28/2013,Buena Vista,$29.64,1.50
3,03/26/2013,Carroll,$22.72,1.00
4,08/17/2017,POLK,$17.25,0.50


In [60]:
df.shape

(2518216, 4)

In [61]:
df[df['date'].isna()].shape

(0, 4)

In [62]:
df[df['county'].isna()].head()

,date,county,sale_dollars,volume_sold_liters
112,07/29/2016,NaN,$89.58,10.50
270,03/02/2016,NaN,$5.67,0.50
301,07/29/2016,NaN,$9.64,0.75
373,04/04/2016,NaN,$166.44,4.50
462,08/10/2016,NaN,$75.60,9.00


In [63]:
df[df['sale_dollars'].isna()].shape

(2, 4)

In [64]:
df[df['volume_sold_liters'].isna()].shape

(0, 4)

## II. Limpar dados: Corrigir, imputar ou remover valores erroneamente inseridos nos conjuntos de dados.

#### - Os dados nan serão removidos pois não é possível recuperar o nome do condado pelo conjunto de dados atual
#### - Realizamos o slugfy para padronizar os nomes dos condados
#### - Colocamos a coluna date em formato date

In [65]:
df = df.dropna()

In [66]:
df['county']=df['county'].apply(slugify)

#### Transformando a coluna data para formato date

In [67]:
df['date'] = pd.to_datetime(df['date'])

In [68]:
df.head()

,date,county,sale_dollars,volume_sold_liters
0,2013-02-26,scott,$8.99,0.38
1,2017-08-22,warren,$287.94,1.75
2,2013-08-28,buena-vista,$29.64,1.50
3,2013-03-26,carroll,$22.72,1.00
4,2017-08-17,polk,$17.25,0.50


In [69]:
counties = df['county'].unique()

#### Procurando por mais inconsistências

##### - Verificamos aqui os Condados com nome parecido utilizando uma biblioteca a qual usa o algoritmo de levenstein para determinar distâncias entre strings
##### - Padronizamos as colunas com valores monetários. Transformamos em float

In [70]:
for a in counties:
    for b in reversed(counties):
        if distance(a,b)==1:
            print(a)

buena-vista
cerro-gordo
cerro-gord
buena-vist
o-brien
obrien


In [71]:
df['sale_dollars'] = df['sale_dollars'].replace({'\$':''}, regex = True).astype(float)

In [72]:
df['volume_sold_liters'] = df['volume_sold_liters'].astype(float)

In [73]:
df.head()

,date,county,sale_dollars,volume_sold_liters
0,2013-02-26,scott,8.99,0.38
1,2017-08-22,warren,287.94,1.75
2,2013-08-28,buena-vista,29.64,1.50
3,2013-03-26,carroll,22.72,1.00
4,2017-08-17,polk,17.25,0.50


#### Utilizar os dados de 2016, pois é o mais recente e tem dados completos

In [74]:
df = df[df['date'].dt.year == 2016]

In [75]:
df.head()

,date,county,sale_dollars,volume_sold_liters
10,2016-06-22,bremer,70.52,4.0
20,2016-12-16,hamilton,90.00,1.2
21,2016-12-22,scott,7.26,1.2
38,2016-01-19,page,376.32,42.0
39,2016-05-18,johnson,156.96,9.0


## III. Construir dados: derivar novos atributos que serão úteis. Por exemplo, derivar o IMC de alguém a partir da altura e peso.

#### - Agrupamos os dados por condado e realizamos as somas dos valores
#### - Depois, coletamos os 5 condados por maior quantidade de litros vendidos

In [76]:
county_group = df.groupby(by=["county"]).sum()

In [77]:
county_group.head()

,sale_dollars,volume_sold_liters
county,,
adair,78170.39,5304.73
adams,16566.53,1207.21
allamakee,146542.63,9515.25
appanoose,148019.33,9113.52
audubon,24182.22,2095.80


In [78]:
most = county_group.nlargest(5, ['volume_sold_liters'])

In [79]:
most.head()

,sale_dollars,volume_sold_liters
county,,
polk,1.110619e+07,600075.43
linn,4.266498e+06,253583.50
scott,3.319892e+06,178965.03
johnson,2.929709e+06,161005.70
black-hawk,2.786531e+06,154994.32


## IV. Integrar dados: criar novos conjuntos de dados combinando dados de várias fontes.

#### - Encontramos um dataset com a estimativa populacional de anos que convergiam com o dataset de vendas de bebidas
#### - Utilizamos 3 colunas: STNAME,CTYNAME,POPESTIMATE, representando respectivamente, estado e população daquele condado
#### - Filtramos pelo estado do IOWA, depois removemos a coluna estado
#### - Padronizamos o nome do condado com regex e slugify

In [80]:
pop_df = pd.read_csv("../data/raw/pop_cty_iowa.csv",usecols=['STNAME','CTYNAME','POPESTIMATE2016'])

In [81]:
pop_df['CTYNAME'] = pop_df['CTYNAME'].replace({' County':''}, regex = True).str.lower()

In [82]:
pop_df = pop_df[pop_df['STNAME']=="Iowa"]

In [84]:
pop_df['CTYNAME']=pop_df['CTYNAME'].apply(slugify)

In [85]:
pop_df['CTYNAME'].unique()

array(['iowa', 'adair', 'adams', 'allamakee', 'appanoose', 'audubon',
       'benton', 'black-hawk', 'boone', 'bremer', 'buchanan',
       'buena-vista', 'butler', 'calhoun', 'carroll', 'cass', 'cedar',
       'cerro-gordo', 'cherokee', 'chickasaw', 'clarke', 'clay',
       'clayton', 'clinton', 'crawford', 'dallas', 'davis', 'decatur',
       'delaware', 'des-moines', 'dickinson', 'dubuque', 'emmet',
       'fayette', 'floyd', 'franklin', 'fremont', 'greene', 'grundy',
       'guthrie', 'hamilton', 'hancock', 'hardin', 'harrison', 'henry',
       'howard', 'humboldt', 'ida', 'jackson', 'jasper', 'jefferson',
       'johnson', 'jones', 'keokuk', 'kossuth', 'lee', 'linn', 'louisa',
       'lucas', 'lyon', 'madison', 'mahaska', 'marion', 'marshall',
       'mills', 'mitchell', 'monona', 'monroe', 'montgomery', 'muscatine',
       'o-brien', 'osceola', 'page', 'palo-alto', 'plymouth',
       'pocahontas', 'polk', 'pottawattamie', 'poweshiek', 'ringgold',
       'sac', 'scott', 'shelby', '

##### - Padronizamos os nomes das colunas e fazemos um merge pelo nome do condado no dataframe most e pop_df

In [87]:
pop_df.rename(columns = {'CTYNAME':'county'}, inplace = True)

In [88]:
df = pd.merge(most, pop_df, how = 'inner', on = 'county')

## V.  Formatar dados: Formatar novamente os dados conforme as necessidades dos modelos.

##### - Modificamos o nome da coluna para padronizar
##### - Realizamos o cálculo de litros por pessoa e dólares por pessoa para saber onde se bebe mais
##### - Criamos valores dummies com as 5 categorias de condados que vendem mais

In [89]:
df.rename(columns = {'POPESTIMATE2016':'pop_estimate_2016'}, inplace = True)

In [90]:
df

,county,sale_dollars,volume_sold_liters,pop_estimate_2016
0,polk,1.110619e+07,600075.43,474277
1,linn,4.266498e+06,253583.50,222188
2,scott,3.319892e+06,178965.03,172135
3,johnson,2.929709e+06,161005.70,146928
4,black-hawk,2.786531e+06,154994.32,133077


In [91]:
df['liters_per_person'] = df_volume['volume_sold_liters']/df_volume['pop_estimate_2016']

In [92]:
df['dollars_per_person'] = df_volume['sale_dollars']/df_volume['pop_estimate_2016']

In [93]:
df = pd.get_dummies(df, columns = ['county'])

In [94]:
df

,sale_dollars,volume_sold_liters,pop_estimate_2016,liters_per_person,dollars_per_person,county_black-hawk,county_johnson,county_linn,county_polk,county_scott
0,1.110619e+07,600075.43,474277,1.265243,23.417090,0,0,0,1,0
1,4.266498e+06,253583.50,222188,1.141302,19.202199,0,0,1,0,0
2,3.319892e+06,178965.03,172135,1.039678,19.286562,0,0,0,0,1
3,2.929709e+06,161005.70,146928,1.095814,19.939757,0,1,0,0,0
4,2.786531e+06,154994.32,133077,1.164697,20.939240,1,0,0,0,0
